In [1]:
# Basic imports
import numpy as np
import matplotlib.pyplot as plt
import gym
import tensorflow as tf
from tensorflow import keras
from datetime import datetime

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _TENSORSHAPEPROTO_

In [3]:
# Import replay buffer and agent
from Simple_DQN_Discrete_Actions import Replay_Buffer, DQN_Agent


## Main

In [46]:
if __name__ == "__main__":
    # Disable eager execution (NOTES)
    tf.compat.v1.disable_eager_execution

    # Create environment
    env = gym.make('LunarLander-v2')

    # Max no. of episodes and steps:
    n_games = 15
    N_steps_max = 50

    # Deep RL hyperparameters:
    learn_rate = 0.001
    gamma = 0.99
    eps_ini = 1.0
    eps_dec = 5e-5
    eps_min = 0.01
    batch_size = 64
    mem_size = 100000
    model_file_name = 'dqn_attempt_0.00.h5'

    # Ini filenames 
    # Total scores
    scores_fname = "LunarLander_Scores.csv"
    # Epsilon
    epsilon_hist_fname = "LunarLander_Epsilon.csv"


    # Create agent
    agent = DQN_Agent(input_dims = env.observation_space.shape, discrete_actions= True, n_actions = env.action_space.n,\
                     learn_rate = learn_rate, gamma=gamma, epsilon=eps_ini, batch_size = batch_size, mem_size = mem_size, \
                     epsilon_dec =eps_dec, epsilon_min =eps_min, dqn_fname = model_file_name)
    print("DQN_Agent created...")

    # Init. score and episode (epsilon) histories
    scores_hist = []
    eps_hist = []

    # Build the DQN (Forgot this on 1st exec...)
    agent.build_DQN(256, 256)

    # Init. timer of main loop
    dqn_begin_time = datetime.now()

    print("Starting main loop...")
    # Loop over episodes:
    for i in range(n_games):
        
        # Init. episode
        done = False
        score = 0.0
        s = env.reset()
        # DEBUG: Fix no. of max steps
        N_steps = 0
        
        # Loop over steps in episodes
        while not done and (N_steps < N_steps_max):
            
            # Choose an action
            a = agent.choose_action(s)
            # Get reward and next state
            s_, r, done, info = env.step(a)
            # Update score
            score += r
            # Store transition
            agent.store_transition(s, a, s_, r, done)
            
            # Learn
            agent.train_dqn(N_steps, i, notify_end_train=True)
            # Update state
            s = s_
            # DEBUG: Increment N_steps
            N_steps += 1
            
        # END while over episode steps
        
        # Update epsilon and score histories
        eps_hist.append(agent.epsilon)
        scores_hist.append(score)

    # END OF MAIN DQN LOOP (i over episodes) 
    print("Exited main loop.")
    
    # Exec time of main loop
    dqn_exec_time = datetime.now()-dqn_begin_time
    print(f"Exec time is {dqn_exec_time}")

DQN_Agent created...
Starting main loop...
Exited main loop.
Exec time is 0:01:47.463107


### Round 1
Let's look at what the agent contains:

In [47]:
agent.memory_buffer.memory_actions

array([3, 1, 3, ..., 0, 0, 0], dtype=int8)

In [6]:
agent.memory_buffer.memory_actions.shape

(100000,)

Let's delve more into the training method:

            # Get minibatch of samples
            # Signature: sample_state, sample_action, sample_new_state, sample_reward, sample_terminal
            #            = Replay_Buffer.sample_buffer(batch_size)
            states, actions, next_states, rewards, dones = self.memory_buffer.sample_buffer(self.batch_size)
            
            # Init. Q(s,a), Q(s', a'), Q*(s,a)            
            q_eval = self.q_network.predict(states)
            q_next = self.q_network.predict(next_states)
            q_target = np.copy(q_eval)
            
            # Ones vector for "dones"
            # IMPORTANT REMARK: This might cause issues, related to Replay_Buffer.store_transition(),
            #                   on line: self.memory_terminal[ind] = int(done) 
            done_ones = np.ones(shape = dones.shape, dtype=np.int8)
            
            # Iterable for minibatch
            batch_index = np.arange(self.batch_size, dtype=np.int32)
            
            # Q-learning update (CRUCIAL)
            q_target[batch_index, actions] = rewards + self.gamma*np.max(q_next, axis = 1)*(done_ones - dones)
            
            # Train network (CRUCIAL)
            self.q_network.train_on_batch(states, q_target)
            
            if ((N_steps+1)% 200)==0 and notify_end_train:
                print(f"Done training at step {N_steps+1} of episode {ep_i+1}")
            
            # Update epsilon for policy
            if self.epsilon_min < self.epsilon:
                self.epsilon -= self.epsilon_dec
            else:
                self.epsilon = self.epsilon_min
            
            return True

In [7]:
batch_size = agent.batch_size
print(f"Batch size = {batch_size}")

Batch size = 64


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
self = agent

In [9]:
##### TRAINING - PART 1 #####
states, actions, next_states, rewards, dones = self.memory_buffer.sample_buffer(self.batch_size)

In [10]:
##### TRAINING - PART 2 #####
# Init. Q(s,a), Q(s', a'), Q*(s,a)            
q_eval = self.q_network.predict(states)
q_next = self.q_network.predict(next_states)
q_target = np.copy(q_eval)
            
# Ones vector for "dones"
done_ones = np.ones(shape = dones.shape, dtype=np.int8)

In [11]:
##### TRAINING - PART 3 #####
#### THIS I DON'T UNDERSTAND ####
# Iterable for minibatch
batch_index = np.arange(self.batch_size, dtype=np.int32)

In [12]:
batch_index

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63])

In [26]:
self.action_space

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[0, 1, 2, 3]

In [14]:
actions

array([0, 3, 3, 1, 1, 2, 1, 1, 0, 2, 1, 1, 2, 0, 1, 1, 0, 3, 0, 1, 2, 2,
       1, 1, 1, 3, 2, 0, 1, 0, 0, 3, 1, 1, 1, 3, 3, 3, 2, 0, 3, 2, 2, 2,
       2, 1, 3, 3, 0, 2, 2, 0, 1, 0, 3, 3, 1, 3, 1, 2, 0, 3, 0, 1],
      dtype=int8)

In [21]:
actions[0:5]

array([0, 3, 3, 1, 1], dtype=int8)

In [15]:
dones

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int8)

In [17]:
done_ones

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int8)

In [19]:
q_target.shape

(64, 4)

In [25]:
q_target[0]

array([74.97974 , 74.20572 , 74.11029 , 74.751236], dtype=float32)

In [23]:
q_target[0,1]

74.20572

In [27]:
batch_index_0 = np.arange(5, dtype=np.int32)

In [28]:
actions_0 = actions[batch_index_0]

In [29]:
actions_0

array([0, 3, 3, 1, 1], dtype=int8)

In [30]:
actions

array([0, 3, 3, 1, 1, 2, 1, 1, 0, 2, 1, 1, 2, 0, 1, 1, 0, 3, 0, 1, 2, 2,
       1, 1, 1, 3, 2, 0, 1, 0, 0, 3, 1, 1, 1, 3, 3, 3, 2, 0, 3, 2, 2, 2,
       2, 1, 3, 3, 0, 2, 2, 0, 1, 0, 3, 3, 1, 3, 1, 2, 0, 3, 0, 1],
      dtype=int8)

In [31]:
q_target[batch_index_0, actions_0]

array([74.97974 , 23.042078, 83.31682 , 21.82932 , 93.63664 ],
      dtype=float32)

In [36]:
q_target_reduced_0 = np.copy(q_target[batch_index_0, actions_0])
for i in range(5):
    for a in self.action_space:
        print(f"q_target[{i},{a}]={q_target[i,a]}")
        #print(f"q_target[batch_index_0, actions_0][{i},{a}] = {q_target_reduced_0[i,a]}\n")

q_target[0,0]=74.979736328125
q_target[0,1]=74.20571899414062
q_target[0,2]=74.11029052734375
q_target[0,3]=74.75123596191406
q_target[1,0]=21.923152923583984
q_target[1,1]=19.79814338684082
q_target[1,2]=18.323339462280273
q_target[1,3]=23.042078018188477
q_target[2,0]=83.1660385131836
q_target[2,1]=82.32978820800781
q_target[2,2]=83.48722839355469
q_target[2,3]=83.31681823730469
q_target[3,0]=23.870349884033203
q_target[3,1]=21.829320907592773
q_target[3,2]=20.331628799438477
q_target[3,3]=24.884305953979492
q_target[4,0]=92.3262939453125
q_target[4,1]=93.63664245605469
q_target[4,2]=85.42981719970703
q_target[4,3]=88.07614135742188


In [38]:
for i in batch_index_0:
    for a in actions_0:
        print(f"q_target[{i},{a}]={q_target[i,a]}")

q_target[0,0]=74.979736328125
q_target[0,3]=74.75123596191406
q_target[0,3]=74.75123596191406
q_target[0,1]=74.20571899414062
q_target[0,1]=74.20571899414062
q_target[1,0]=21.923152923583984
q_target[1,3]=23.042078018188477
q_target[1,3]=23.042078018188477
q_target[1,1]=19.79814338684082
q_target[1,1]=19.79814338684082
q_target[2,0]=83.1660385131836
q_target[2,3]=83.31681823730469
q_target[2,3]=83.31681823730469
q_target[2,1]=82.32978820800781
q_target[2,1]=82.32978820800781
q_target[3,0]=23.870349884033203
q_target[3,3]=24.884305953979492
q_target[3,3]=24.884305953979492
q_target[3,1]=21.829320907592773
q_target[3,1]=21.829320907592773
q_target[4,0]=92.3262939453125
q_target[4,3]=88.07614135742188
q_target[4,3]=88.07614135742188
q_target[4,1]=93.63664245605469
q_target[4,1]=93.63664245605469


In [40]:
print(q_next[batch_index_0])

[[75.9446   75.16475  75.147255 75.732574]
 [23.100016 20.98738  19.517458 24.180845]
 [84.673706 83.893295 85.082504 84.81499 ]
 [23.770245 21.665264 20.215914 24.82218 ]
 [94.46664  95.757164 87.80471  90.31445 ]]


In [41]:
np.max(q_next[batch_index_0], axis = 1)

array([75.9446  , 24.180845, 85.082504, 24.82218 , 95.757164],
      dtype=float32)

In [42]:
q_next_0 = np.copy(q_next[batch_index_0])

In [43]:
q_next_0

array([[75.9446  , 75.16475 , 75.147255, 75.732574],
       [23.100016, 20.98738 , 19.517458, 24.180845],
       [84.673706, 83.893295, 85.082504, 84.81499 ],
       [23.770245, 21.665264, 20.215914, 24.82218 ],
       [94.46664 , 95.757164, 87.80471 , 90.31445 ]], dtype=float32)

In [44]:
q_next_0[2][3]

84.81499

In [45]:
##### TRAINING - PART 4 #####
#### THIS I DON'T UNDERSTAND ####

# Q-learning update (CRUCIAL)
# Looks like this is done to avoid a loop
q_target[batch_index, actions] = rewards + self.gamma*np.max(q_next, axis = 1)*(done_ones - dones)

In [ ]:
##### TRAINING - PART 5 #####
# Train network (CRUCIAL)
self.q_network.train_on_batch(states, q_target)

### Round 2
Here, I'm trying to unpack the line:

            q_target[batch_index, actions] = rewards + self.gamma*np.max(q_next, axis = 1)*(done_ones - dones)
            
The batch_index variable is just the range [0:63] in array form, and this line replaces a loop over the minibatch of sampled actions.

In [54]:
self = agent

In [55]:
agent.memory_buffer.memory_actions

array([3, 1, 3, ..., 0, 0, 0], dtype=int8)

In [49]:
##### TRAINING - PART 1 #####
states, actions, next_states, rewards, dones = self.memory_buffer.sample_buffer(self.batch_size)

In [50]:
##### TRAINING - PART 2 #####
# Init. Q(s,a), Q(s', a'), Q*(s,a)            
q_eval = self.q_network.predict(states)
q_next = self.q_network.predict(next_states)
q_target = np.copy(q_eval)
            
# Ones vector for "dones"
done_ones = np.ones(shape = dones.shape, dtype=np.int8)

In [51]:
##### TRAINING - PART 3 #####
#### THIS I DON'T UNDERSTAND ####
# Iterable for minibatch
batch_index = np.arange(self.batch_size, dtype=np.int32)

In [52]:
batch_index

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63])

In [26]:
self.action_space

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[0, 1, 2, 3]

In [53]:
batch_indices = [i for i in range(self.batch_size)]

In [56]:
batch_indices

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63]

In [65]:
# create 2 copies to compare main Q-update
q_target_v1 = np.copy(q_target)
q_target_v2 = np.copy(q_target)

In [66]:
q_target_v1[batch_index, actions] = rewards + self.gamma*np.max(q_next, axis = 1)*(done_ones - dones)

In [67]:
q_target_v1[batch_indices, actions]

array([46388.316, 75452.46 , 48130.96 , 68365.695, 41415.332, 74317.22 ,
       41382.363, 48550.54 , 78040.43 , 48479.992, 32507.139, 75191.59 ,
       43172.996, 74218.3  , 75578.11 , 59105.23 , 67476.98 , 48761.234,
       53650.516, 80775.59 , 61559.105, 66108.74 , 74058.93 , 82146.61 ,
       72897.54 , 72940.13 , 68213.93 , 76570.62 , 57692.664, 72257.47 ,
       70827.03 , 72759.21 , 45532.234, 37226.1  , 36616.703, 36058.297,
       74096.12 , 76208.77 , 78330.484, 73829.88 , 44656.492, 68822.86 ,
       71652.84 , 48412.934, 28663.545, 63212.71 , 62622.832, 74589.97 ,
       60131.29 , 45705.39 , 43078.977, 26622.963, 49164.816, 42615.363,
       54385.957, 53758.766, 47278.168, 27548.688, 67266.55 , 78435.805,
       66369.52 , 76329.14 , 66047.25 , 66537.83 ], dtype=float32)

In [68]:
q_target_v1[batch_index, actions]

array([46388.316, 75452.46 , 48130.96 , 68365.695, 41415.332, 74317.22 ,
       41382.363, 48550.54 , 78040.43 , 48479.992, 32507.139, 75191.59 ,
       43172.996, 74218.3  , 75578.11 , 59105.23 , 67476.98 , 48761.234,
       53650.516, 80775.59 , 61559.105, 66108.74 , 74058.93 , 82146.61 ,
       72897.54 , 72940.13 , 68213.93 , 76570.62 , 57692.664, 72257.47 ,
       70827.03 , 72759.21 , 45532.234, 37226.1  , 36616.703, 36058.297,
       74096.12 , 76208.77 , 78330.484, 73829.88 , 44656.492, 68822.86 ,
       71652.84 , 48412.934, 28663.545, 63212.71 , 62622.832, 74589.97 ,
       60131.29 , 45705.39 , 43078.977, 26622.963, 49164.816, 42615.363,
       54385.957, 53758.766, 47278.168, 27548.688, 67266.55 , 78435.805,
       66369.52 , 76329.14 , 66047.25 , 66537.83 ], dtype=float32)

In [71]:
for i in batch_indices:
    q_next_i = q_next[i]
    q_target_v2[i,actions[i]] = rewards[i]+self.gamma*np.max(q_next_i)*(done_ones[i]-dones[i])
        

In [72]:
q_target_v1 == q_target_v2

array([[ True,  True,  True,  True],
       [False,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True, False,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [False,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
 

In [73]:
q_target_v1[1,0]

75452.46

In [75]:
q_target_v2[1,0]

75452.45

In [76]:
for i in batch_indices:
    for j in self.action_space:
        if q_target_v1[i,j] != q_target_v2[i,j]:
            print(f"q_target_v1[{i},{j}] not equal q_target_v2[{i},{j}]\n")
            print(f"q_target_v1[{i},{j}]={q_target_v1[i,j]}\n")
            print(f"q_target_v2[{i},{j}]={q_target_v2[i,j]}\n")
            print(f"-----------------------------------------------------")
            

q_target_v1[1,0] not equal q_target_v2[1,0]

q_target_v1[1,0]=75452.4609375

q_target_v2[1,0]=75452.453125

-----------------------------------------------------
q_target_v1[8,2] not equal q_target_v2[8,2]

q_target_v1[8,2]=78040.4296875

q_target_v2[8,2]=78040.421875

-----------------------------------------------------
q_target_v1[14,0] not equal q_target_v2[14,0]

q_target_v1[14,0]=75578.109375

q_target_v2[14,0]=75578.1015625

-----------------------------------------------------
q_target_v1[34,1] not equal q_target_v2[34,1]

q_target_v1[34,1]=36616.703125

q_target_v2[34,1]=36616.69921875

-----------------------------------------------------
q_target_v1[35,2] not equal q_target_v2[35,2]

q_target_v1[35,2]=36058.296875

q_target_v2[35,2]=36058.30078125

-----------------------------------------------------
q_target_v1[38,1] not equal q_target_v2[38,1]

q_target_v1[38,1]=78330.484375

q_target_v2[38,1]=78330.4765625

-----------------------------------------------------
q_target_v

In [77]:
help(np.around)

Help on function around in module numpy:

around(a, decimals=0, out=None)
    Evenly round to the given number of decimals.
    
    Parameters
    ----------
    a : array_like
        Input data.
    decimals : int, optional
        Number of decimal places to round to (default: 0).  If
        decimals is negative, it specifies the number of positions to
        the left of the decimal point.
    out : ndarray, optional
        Alternative output array in which to place the result. It must have
        the same shape as the expected output, but the type of the output
        values will be cast if necessary. See :ref:`ufuncs-output-type` for more
        details.
    
    Returns
    -------
    rounded_array : ndarray
        An array of the same type as `a`, containing the rounded values.
        Unless `out` was specified, a new array is created.  A reference to
        the result is returned.
    
        The real and imaginary parts of complex numbers are rounded
        separa

In [70]:
actions

array([0, 0, 1, 3, 1, 0, 2, 2, 2, 1, 3, 3, 0, 2, 0, 1, 1, 1, 2, 2, 2, 0,
       3, 2, 0, 3, 0, 2, 0, 0, 1, 0, 2, 2, 1, 2, 2, 3, 1, 0, 3, 2, 1, 0,
       3, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 3, 0, 0, 0, 0, 2],
      dtype=int8)

In [69]:
q_target_v1

array([[46388.316, 46647.812, 45403.855, 45254.984],
       [75452.46 , 75972.86 , 73949.734, 73707.625],
       [46272.797, 48130.96 , 45835.105, 45684.79 ],
       [68706.83 , 69919.88 , 68057.78 , 68365.695],
       [40130.203, 41415.332, 39749.87 , 39619.566],
       [74317.22 , 74446.04 , 72464.16 , 72226.88 ],
       [41880.965, 42621.156, 41382.363, 41348.36 ],
       [49125.35 , 49993.527, 48550.54 , 48500.742],
       [77940.64 , 79316.59 , 78040.43 , 76951.96 ],
       [47083.82 , 48479.992, 46638.06 , 46485.16 ],
       [33429.867, 34024.74 , 33107.12 , 32507.139],
       [75252.89 , 76581.39 , 74542.19 , 75191.59 ],
       [43172.996, 43620.05 , 42456.363, 42317.195],
       [72917.67 , 74204.99 , 74218.3  , 71992.336],
       [75578.11 , 76128.016, 74100.695, 73858.09 ],
       [57904.043, 59105.23 , 57356.223, 57168.617],
       [65522.867, 67476.98 , 64903.863, 64691.305],
       [47265.938, 48761.234, 46818.51 , 46664.99 ],
       [53688.305, 54636.41 , 53650.516, 53006

In [61]:
q_target.shape

(64, 4)

In [45]:
##### TRAINING - PART 4 #####
#### THIS I DON'T UNDERSTAND ####

# Q-learning update (CRUCIAL)
# Looks like this is done to avoid a loop
q_target[batch_index, actions] = rewards + self.gamma*np.max(q_next, axis = 1)*(done_ones - dones)

In [ ]:
##### TRAINING - PART 5 #####
# Train network (CRUCIAL)
self.q_network.train_on_batch(states, q_target)